# Pipeline: Echocardiogram Analysis

## Manual pipeline for DCM download, decompress and classification of views

In [1]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [2]:
import os
import sys
import pandas as pd
import datetime

projectdir = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(projectdir, 'src'))

import d00_utils.db_utils as db
from d02_intermediate.download_dcm import s3_download_decomp_dcm, dcmdir_to_jpgs_for_classification
#from d03_classification.predict_views import run_classify, agg_probabilities, predict_views

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
table_name='instances_w_labels'
train_test_ratio=0.5 
downsample_ratio=0.001
train=False
dcm_path = os.path.join(os.path.expanduser('~/data'),'01_raw')
model_path = os.path.join(os.path.expanduser('~/models'), 'view_23_e5_class_11-Mar-2018')

In [4]:
#dir_name = s3_download_decomp_dcm(dcm_path, table_name, train_test_ratio, downsample_ratio, train)
dir_name = 'test_split50_downsampleby1000/raw'

In [5]:
dcm_dir = os.path.join(dcm_path, dir_name)
img_dir = os.path.join(os.path.expanduser('~/data'),'02_intermediate', dir_name)

#dcmdir_to_jpgs_for_classification(dcm_dir, img_dir)

In [ ]:
run_classify(img_dir, model_path, if_exists='append', feature_dim=1)

In [ ]:
agg_probabilities(if_exists='append')

In [ ]:
predict_views(if_exists='append')

In [ ]:
io_class = db.dbReadWriteClassification()
io_class.list_tables()

In [ ]:
io_class.get_table('predictions')

In [ ]:
evaluate_views(dir_name, model_name, datetime.date.today(), study_filter=None, if_exists="append") 

In [ ]:
io_class.get_table('evaluations')

# Manual pipeline for the segmentation of image files

In [6]:
from d02_intermediate.create_seg_view import create_seg_view 
from d04_segmentation.segment_view import run_segment
from d04_segmentation.generate_masks import generate_masks
from d04_segmentation.evaluate_masks import evaluate_masks

SyntaxError: invalid syntax (meas_utils.py, line 9)

In [24]:
dcm_dir = os.path.join(dcm_path, dir_name)
dcm_dir

'/home/ubuntu/data/01_raw/test_split50_downsampleby1000/raw'

In [7]:
model_path = os.path.join(os.path.expanduser('~/models'))
run_segment(dcm_dir, model_path)

Number of files in the directory: 26
Number of files successfully matched with predictions: 156
DCM PATH : /home/ubuntu/data/01_raw/test_split50_downsampleby1000
a_118052_6S0RC5F1.dcm
for a4c
DCM/dicom PATH : /home/ubuntu/data/01_raw/test_split50_downsampleby1000
a_118052_6S0RC5F1.dcm /home/ubuntu/data/01_raw/test_split50_downsampleby1000

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.conv2d_transpose instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_e

KeyboardInterrupt: 

In [ ]:
create_seg_view()

In [ ]:
io_views = db.dbReadWriteViews()
io_views.list_tables()

In [ ]:
io_views.get_table('frames_by_volume_mask').head()

In [ ]:
io_views.get_table('chords_by_volume_mask').head()

In [35]:
generate_masks(dcm_dir)

gdcmdump /home/ubuntu/data/01_raw/test_split50_downsampleby1000/raw/a_117839_6R3ORWX8.dcm_raw
['# Dicom-File-Format', '', '# Dicom-Meta-Information-Header', '# Used TransferSyntax: ', '(0002,0000) UL 240                                                # 4,1 File Meta Information Group Length', '(0002,0001) OB 00\\01                                              # 2,1 File Meta Information Version', '(0002,0002) UI [1.2.840.10008.5.1.4.1.1.3.1]                      # 28,1 Media Storage SOP Class UID', '(0002,0003) UI [1.2.826.0.1.3680043.2.1143.5022032701495910660291471839593030908]         # 64,1 Media Storage SOP Instance UID', '(0002,0010) UI [1.2.840.10008.1.2.1]                              # 20,1 Transfer Syntax UID', '(0002,0012) UI [1.2.826.0.1.3680043.2.1143.107.104.103.115.2.6.3]         # 48,1 Implementation Class UID', '(0002,0013) SH [GDCM 2.6.3]                                       # 10,1 Implementation Version Name', '(0002,0016) AE [gdcmconv]                              

['# Dicom-File-Format', '', '# Dicom-Meta-Information-Header', '# Used TransferSyntax: ', '(0002,0000) UL 240                                                # 4,1 File Meta Information Group Length', '(0002,0001) OB 00\\01                                              # 2,1 File Meta Information Version', '(0002,0002) UI [1.2.840.10008.5.1.4.1.1.3.1]                      # 28,1 Media Storage SOP Class UID', '(0002,0003) UI [1.2.826.0.1.3680043.2.1143.5022032701495910660291471839593030908]         # 64,1 Media Storage SOP Instance UID', '(0002,0010) UI [1.2.840.10008.1.2.1]                              # 20,1 Transfer Syntax UID', '(0002,0012) UI [1.2.826.0.1.3680043.2.1143.107.104.103.115.2.6.3]         # 48,1 Implementation Class UID', '(0002,0013) SH [GDCM 2.6.3]                                       # 10,1 Implementation Version Name', '(0002,0016) AE [gdcmconv]                                         # 8,1 Source Application Entity Title', '', '# Dicom-Data-Set', '# Used TransferSyn

['# Dicom-File-Format', '', '# Dicom-Meta-Information-Header', '# Used TransferSyntax: ', '(0002,0000) UL 240                                                # 4,1 File Meta Information Group Length', '(0002,0001) OB 00\\01                                              # 2,1 File Meta Information Version', '(0002,0002) UI [1.2.840.10008.5.1.4.1.1.3.1]                      # 28,1 Media Storage SOP Class UID', '(0002,0003) UI [1.2.826.0.1.3680043.2.1143.2942016324660085870209326381248645371]         # 64,1 Media Storage SOP Instance UID', '(0002,0010) UI [1.2.840.10008.1.2.1]                              # 20,1 Transfer Syntax UID', '(0002,0012) UI [1.2.826.0.1.3680043.2.1143.107.104.103.115.2.6.3]         # 48,1 Implementation Class UID', '(0002,0013) SH [GDCM 2.6.3]                                       # 10,1 Implementation Version Name', '(0002,0016) AE [gdcmconv]                                         # 8,1 Source Application Entity Title', '', '# Dicom-Data-Set', '# Used TransferSyn

['# Dicom-File-Format', '', '# Dicom-Meta-Information-Header', '# Used TransferSyntax: ', '(0002,0000) UL 240                                                # 4,1 File Meta Information Group Length', '(0002,0001) OB 00\\01                                              # 2,1 File Meta Information Version', '(0002,0002) UI [1.2.840.10008.5.1.4.1.1.3.1]                      # 28,1 Media Storage SOP Class UID', '(0002,0003) UI [1.2.826.0.1.3680043.2.1143.2942016324660085870209326381248645371]         # 64,1 Media Storage SOP Instance UID', '(0002,0010) UI [1.2.840.10008.1.2.1]                              # 20,1 Transfer Syntax UID', '(0002,0012) UI [1.2.826.0.1.3680043.2.1143.107.104.103.115.2.6.3]         # 48,1 Implementation Class UID', '(0002,0013) SH [GDCM 2.6.3]                                       # 10,1 Implementation Version Name', '(0002,0016) AE [gdcmconv]                                         # 8,1 Source Application Entity Title', '', '# Dicom-Data-Set', '# Used TransferSyn

['# Dicom-File-Format', '', '# Dicom-Meta-Information-Header', '# Used TransferSyntax: ', '(0002,0000) UL 240                                                # 4,1 File Meta Information Group Length', '(0002,0001) OB 00\\01                                              # 2,1 File Meta Information Version', '(0002,0002) UI [1.2.840.10008.5.1.4.1.1.3.1]                      # 28,1 Media Storage SOP Class UID', '(0002,0003) UI [1.2.826.0.1.3680043.2.1143.5506369456283033536077401289886011113]         # 64,1 Media Storage SOP Instance UID', '(0002,0010) UI [1.2.840.10008.1.2.1]                              # 20,1 Transfer Syntax UID', '(0002,0012) UI [1.2.826.0.1.3680043.2.1143.107.104.103.115.2.6.3]         # 48,1 Implementation Class UID', '(0002,0013) SH [GDCM 2.6.3]                                       # 10,1 Implementation Version Name', '(0002,0016) AE [gdcmconv]                                         # 8,1 Source Application Entity Title', '', '# Dicom-Data-Set', '# Used TransferSyn

...

['# Dicom-File-Format', '', '# Dicom-Meta-Information-Header', '# Used TransferSyntax: ', '(0002,0000) UL 240                                                # 4,1 File Meta Information Group Length', '(0002,0001) OB 00\\01                                              # 2,1 File Meta Information Version', '(0002,0002) UI [1.2.840.10008.5.1.4.1.1.3.1]                      # 28,1 Media Storage SOP Class UID', '(0002,0003) UI [1.2.826.0.1.3680043.2.1143.4190111634275456749237473551464008443]         # 64,1 Media Storage SOP Instance UID', '(0002,0010) UI [1.2.840.10008.1.2.1]                              # 20,1 Transfer Syntax UID', '(0002,0012) UI [1.2.826.0.1.3680043.2.1143.107.104.103.115.2.6.3]         # 48,1 Implementation Class UID', '(0002,0013) SH [GDCM 2.6.3]                                       # 10,1 Implementation Version Name', '(0002,0016) AE [gdcmconv]                                         # 8,1 Source Application Entity Title', '', '# Dicom-Data-Set', '# Used TransferSyn

['# Dicom-File-Format', '', '# Dicom-Meta-Information-Header', '# Used TransferSyntax: ', '(0002,0000) UL 240                                                # 4,1 File Meta Information Group Length', '(0002,0001) OB 00\\01                                              # 2,1 File Meta Information Version', '(0002,0002) UI [1.2.840.10008.5.1.4.1.1.3.1]                      # 28,1 Media Storage SOP Class UID', '(0002,0003) UI [1.2.826.0.1.3680043.2.1143.4190111634275456749237473551464008443]         # 64,1 Media Storage SOP Instance UID', '(0002,0010) UI [1.2.840.10008.1.2.1]                              # 20,1 Transfer Syntax UID', '(0002,0012) UI [1.2.826.0.1.3680043.2.1143.107.104.103.115.2.6.3]         # 48,1 Implementation Class UID', '(0002,0013) SH [GDCM 2.6.3]                                       # 10,1 Implementation Version Name', '(0002,0016) AE [gdcmconv]                                         # 8,1 Source Application Entity Title', '', '# Dicom-Data-Set', '# Used TransferSyn

['# Dicom-File-Format', '', '# Dicom-Meta-Information-Header', '# Used TransferSyntax: ', '(0002,0000) UL 240                                                # 4,1 File Meta Information Group Length', '(0002,0001) OB 00\\01                                              # 2,1 File Meta Information Version', '(0002,0002) UI [1.2.840.10008.5.1.4.1.1.3.1]                      # 28,1 Media Storage SOP Class UID', '(0002,0003) UI [1.2.826.0.1.3680043.2.1143.1381475875002953482324038660716091948]         # 64,1 Media Storage SOP Instance UID', '(0002,0010) UI [1.2.840.10008.1.2.1]                              # 20,1 Transfer Syntax UID', '(0002,0012) UI [1.2.826.0.1.3680043.2.1143.107.104.103.115.2.6.3]         # 48,1 Implementation Class UID', '(0002,0013) SH [GDCM 2.6.3]                                       # 10,1 Implementation Version Name', '(0002,0016) AE [gdcmconv]                                         # 8,1 Source Application Entity Title', '', '# Dicom-Data-Set', '# Used TransferSyn

['# Dicom-File-Format', '', '# Dicom-Meta-Information-Header', '# Used TransferSyntax: ', '(0002,0000) UL 240                                                # 4,1 File Meta Information Group Length', '(0002,0001) OB 00\\01                                              # 2,1 File Meta Information Version', '(0002,0002) UI [1.2.840.10008.5.1.4.1.1.3.1]                      # 28,1 Media Storage SOP Class UID', '(0002,0003) UI [1.2.826.0.1.3680043.2.1143.2364173200108721438626100302770678323]         # 64,1 Media Storage SOP Instance UID', '(0002,0010) UI [1.2.840.10008.1.2.1]                              # 20,1 Transfer Syntax UID', '(0002,0012) UI [1.2.826.0.1.3680043.2.1143.107.104.103.115.2.6.3]         # 48,1 Implementation Class UID', '(0002,0013) SH [GDCM 2.6.3]                                       # 10,1 Implementation Version Name', '(0002,0016) AE [gdcmconv]                                         # 8,1 Source Application Entity Title', '', '# Dicom-Data-Set', '# Used TransferSyn

,index,file_name,file_path,studyidk,instanceidk,indexinmglist,x1coordinate,y1coordinate,x2coordinate,y2coordinate,chamber,frame,view,instancefilename,lines,points,mask
0,0,6R3ORWX8,/home/ubuntu/data/01_raw/test_split50_downsamp...,117839,2756757,0,"[407.0, 467.94762516046205, 471.585000826856, ...","[355.0, 378.66489088575105, 393.412005953365, ...","[306.0, 441.436249193028, 400.30874757908305, ...","[537.0, 363.952533892834, 353.85760167850196, ...",la,50,A4C,6R3ORWX8,"[((407.0, 355.0), (306.0, 537.0)), ((467.94762...","[(407.0, 355.0), (467.94762516046205, 378.6648...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,1,1FZH4296,/home/ubuntu/data/01_raw/test_split50_downsamp...,74920,1507806,0,"[411.0, 359.26647785787804, 339.010515165031, ...","[455.0, 467.706591143151, 458.830608831401, 44...","[326.0, 488.75926553672303, 500.85333787465896...","[182.0, 427.388323917137, 408.43998637602203, ...",lv,51,A4C,1FZH4296,"[((411.0, 455.0), (326.0, 182.0)), ((359.26647...","[(411.0, 455.0), (359.26647785787804, 467.7065...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,2,1FZH4296,/home/ubuntu/data/01_raw/test_split50_downsamp...,74920,1507806,1,"[399.0, 341.228424657534, 327.664457831325, 32...","[420.0, 432.240753424657, 423.661144578313, 41...","[339.0, 448.339024390244, 466.187804878049, 46...","[195.0, 403.677926829268, 386.721585365854, 37...",lv,71,A4C,1FZH4296,"[((399.0, 420.0), (339.0, 195.0)), ((341.22842...","[(399.0, 420.0), (341.228424657534, 432.240753...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,3,6OCGGRPY,/home/ubuntu/data/01_raw/test_split50_downsamp...,117506,2747188,0,"[400.0, 348.138157894737, 344.276466992665, 33...","[349.0, 350.042105263158, 338.139486552567, 32...","[377.0, 451.16655632363, 452.31828449031195, 4...","[108.0, 340.20951952098096, 327.82844172913997...",lv,45,A4C,6OCGGRPY,"[((400.0, 349.0), (377.0, 108.0)), ((348.13815...","[(400.0, 349.0), (348.138157894737, 350.042105...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,4,6OCGGRPY,/home/ubuntu/data/01_raw/test_split50_downsamp...,117506,2747188,1,"[388.0, 341.063307984791, 339.403916960439, 33...","[324.0, 323.105513307985, 313.024481002742, 30...","[374.0, 432.517260712855, 434.166920304365, 43...","[122.0, 316.767120544654, 306.456748097717, 29...",lv,63,A4C,6OCGGRPY,"[((388.0, 324.0), (374.0, 122.0)), ((341.06330...","[(388.0, 324.0), (341.063307984791, 323.105513...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
5,5,6OCGGRPY,/home/ubuntu/data/01_raw/test_split50_downsamp...,117506,2747188,2,"[384.0, 356.46368852459, 383.39106557377096, 4...","[336.0, 334.648852459016, 335.946557377049, 33...","[411.0, 341.844532908705, 339.533598726115, 33...","[462.0, 337.78152866242, 345.344585987261, 352...",la,63,A4C,6OCGGRPY,"[((384.0, 336.0), (411.0, 462.0)), ((356.46368...","[(384.0, 336.0), (356.46368852459, 334.6488524...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
6,6,6HWX6ND4,/home/ubuntu/data/01_raw/test_split50_downsamp...,116770,2725223,0,"[438.0, 414.43819809069197, 371.70709843520103...","[409.0, 408.24214797136, 397.72000468102203, 3...","[429.0, 504.95603255814, 510.868097674419, 514...","[171.0, 404.81920465116303, 392.457613953488, ...",lv,1,A4C,6HWX6ND4,"[((438.0, 409.0), (429.0, 171.0)), ((414.43819...","[(438.0, 409.0), (414.43819809069197, 408.2421...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
7,7,6HWX6ND4,/home/ubuntu/data/01_raw/test_split50_downsamp...,116770,2725223,1,"[423.0, 429.42800000000005, 375.869011725293, ...","[382.0, 383.436, 371.594304857621, 362.0653266...","[431.0, 471.94310344827596, 473.48956743002503...","[198.0, 385.284482758621, 375.838676844784, 36...",lv,26,A4C,6HWX6ND4,"[((423.0, 382.0), (431.0, 198.0)), ((429.42800...","[(423.0, 382.0), (429.42800000000005, 383.436)...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
8,8,6HWX6ND4,/home/ubuntu/data/01_raw/test_split50_downsamp...,116770,2725223,2,"[428.0, 459.056144067797, 485.693855932203, 48..."

In [ ]:
io_segmentation = db.dbReadWriteSegmentations()
io_segmentation.list_tables()

In [ ]:
evaluate_masks()

In [18]:
io_views = db.dbReadWriteViews()

chords_by_volume_mask_df = io_views.get_table("chords_by_volume_mask")
#instances_w_labels_test_downsampleby5_df = io_views.get_table('instances_w_labels_test_downsampleby5')    
#chords_by_volume_mask_df.loc[chords_by_volume_mask_df["view_name"].str.contains('ven'), "chamber"] = "lv"
#chords_by_volume_mask_df.loc[chords_by_volume_mask_df["view_name"].str.contains('atr'), "chamber"] = "la"

#merge_df = pd.merge(instances_w_labels_test_downsampleby5_df, chords_by_volume_mask_df, 
#                  how='inner', on=['studyidk', 'instanceidk'])

#merge_df = chords_by_volume_mask_df[chords_by_volume_mask_df['instanceidk'].isin(instance_ids)]

#start = time()
group_df = chords_by_volume_mask_df.groupby(["studyidk", "instanceidk", "indexinmglist"]).agg(
    {
        "x1coordinate": list,
        "y1coordinate": list,
        "x2coordinate": list,
        "y2coordinate": list,
        "chamber": pd.Series.unique,
        "frame": pd.Series.unique,
        "view" : pd.Series.unique,
        "instancefilename" : pd.Series.unique,
    }
)
#end = time()
print(f"{int(80)} seconds to group {len(group_df)} rows")


80 seconds to group 77442 rows


In [19]:
group_df

x1coordinate  \
studyidk instanceidk indexinmglist                                                      
2882.0   57164       0              [422.0, 408.7043904209, 383.18264488286104, 32...   
                     1              [445.0, 406.617336371168, 354.98270447906503, ...   
2966.0   58207       0              [350.578541245637, 393.272428782834, 381.88757...   
                     1              [360.0, 402.051138189609, 380.153414568827, 34...   
5898.0   102950      0              [302.0, 244.70478260869598, 238.836057023644, ...   
...                                                                               ...   
         3003846     2              [398.127159827214, 398.375647948164, 398.62413...   
         3004262     1              [285.0, 322.77752293578, 344.253283302064, 347...   
         3020373     1              [297.0, 259.991770299927, 305.975310899781, 35...   
         3121715     0              [428.0, 456.497120230382, 363.491360691145, 34...   
                     1              [422.0, 440.2, 355.003305785124, 347.743801652...   

                                                                         y1coordinate  \
studyidk instanceidk indexinmglist                                                      
2882.0   57164       0              [306.0, 308.185087082729, 311.836528976572, 33...   
                     1              [306.0, 325.03537696768797, 345.54073758519996...   
2966.0   58207       0              [336.381591034356, 364.54138919718804, 357.032...   
                     1              [347.0, 359.631119442957, 352.89335832887, 350...   
5898.0   102950      0              [374.0, 363.26195652173897, 349.431675938804, ...   
...                                                                               ...   
         3003846     2              [393.89956803455703, 401.519870410367, 409.140...   
         3004262     1              [316.0, 319.738532110092, 328.478189493433, 33...   
         3020373     1              [308.0, 301.990307242136, 309.970921726408, 31...   
         3121715     0              [470.0, 471.829865610751, 467.489596832253, 45...   
                     1              [428.0, 428.0, 426.44958677686003, 415.1570247...   

                                                                         x2coordinate  \
studyidk instanceidk indexinmglist                                                      
2882.0   57164       0              [297.0, 426.667431561997, 422.381486742424, 41...   
                     1              [312.0, 447.06858778625997, 448.0, 448.0, 446....   
2966.0   58207       0              [282.063809967396, 423.656674208145, 434.86361...   
                     1              [285.0, 440.0, 440.17644298866895, 442.1610559...   
5898.0   102950      0              [342.0, 321.691810344828, 373.83403090792, 382...   
...                                                                               ...   
         3003846     2              [262.795847750865, 265.995833333333, 272.3875,...   
         3004262     1              [289.0, 259.092741935484, 236.789603960396, 23...   
         3020373     1              [308.0, 229.49172081324699, 223.394047369524, ...   
         3121715     0              [387.0, 504.970785659802, 508.912356979405, 51...   
                     1              [393.0, 488.726763717805, 488.18029115341596, ...   

                                                                         y2coordinate  \
studyidk instanceidk indexinmglist                                                      
2882.0   57164       0              [120.0, 296.113150831991, 285.493221275253, 27...   
                     1              [109.0, 297.725648854962, 282.742258883249, 26...   
2966.0   58207       0              [172.76432231020001, 351.817986425339, 334.848...   
                     1              [124.0, 346.86804932735396, 332.706241147309, ...   
5898.0   102950      0              [128.0, 375.780172413793, 371

In [20]:

path = dcm_dir

file_paths = []
filenames = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.dcm' in file:
            file_paths.append(os.path.join(r, file))
            fullfilename = os.path.basename(os.path.join(r, file))
            #print(str(fullfilename).split('.')[0])
            f = str(fullfilename).split('.')[0]
            f = str(f).split('_')[2]
            #f = str(fullfilename).split('.')[0]
            filenames.append(f)

print("Number of files in the directory: {}".format(len(file_paths)))
#print(filenames)
filename_df = pd.DataFrame({'file_name': filenames})
filename_df['file_path'] = path
print(filename_df.head())

group_df = group_df.reset_index()

Number of files in the directory: 26
  file_name                                          file_path
0  6GZUJIRW  /home/ubuntu/data/01_raw/test_split50_downsamp...
1  56LHSUYT  /home/ubuntu/data/01_raw/test_split50_downsamp...
2  6S0RC5F1  /home/ubuntu/data/01_raw/test_split50_downsamp...
3  44GNOLB7  /home/ubuntu/data/01_raw/test_split50_downsamp...
4  1FZH4296  /home/ubuntu/data/01_raw/test_split50_downsamp...


In [21]:
file_gt_masks = pd.merge(filename_df, group_df, how='inner', left_on =['file_name'], right_on = ['instancefilename'])
print("Number of files successfully matched with ground truth masks: {}".format(file_gt_masks.shape[0]))

Number of files successfully matched with ground truth masks: 22


In [22]:
file_gt_masks.head()

,file_name,file_path,studyidk,instanceidk,indexinmglist,x1coordinate,y1coordinate,x2coordinate,y2coordinate,chamber,frame,view,instancefilename
0,6S0RC5F1,/home/ubuntu/data/01_raw/test_split50_downsamp...,118052,2764775,0,"[435.0, 418.481231140616, 467.443693421847, 47...","[354.0, 361.953590826795, 339.860772480386, 34...","[508.0, 394.27413277180494, 394.822398315416, ...","[471.0, 377.05716502272, 385.171495068159, 393...",la,64,A4C,6S0RC5F1
1,6S0RC5F1,/home/ubuntu/data/01_raw/test_split50_downsamp...,118052,2764775,0,"[435.0, 418.481231140616, 467.443693421847, 47...","[354.0, 361.953590826795, 339.860772480386, 34...","[508.0, 394.27413277180494, 394.822398315416, ...","[471.0, 377.05716502272, 385.171495068159, 393...",la,64,A4C,6S0RC5F1
2,44GNOLB7,/home/ubuntu/data/01_raw/test_split50_downsamp...,96997,2159081,0,"[400.0, 459.201908023483, 461.0, 461.0, 458.72...","[366.0, 374.807632093933, 390.481050955414, 40...","[283.0, 447.38311965812005, 428.14935897435896...","[523.0, 366.0, 366.0, 366.0, 366.0, 366.0, 366...",la,22,A4C,44GNOLB7
3,44GNOLB7,/home/ubuntu/data/01_raw/test_split50_downsamp...,96997,2159081,0,"[400.0, 459.201908023483, 461.0, 461.0, 458.72...","[366.0, 374.807632093933, 390.481050955414, 40...","[283.0, 447.38311965812005, 428.14935897435896...","[523.0, 366.0, 366.0, 366.0, 366.0, 366.0, 366...",la,22,A4C,44GNOLB7
4,1FZH4296,/home/ubuntu/data/01_raw/test_split50_downsamp...,74920,1507806,0,"[411.0, 359.26647785787804, 339.010515165031, ...","[455.0, 467.706591143151, 458.830608831401, 44...","[326.0, 488.75926553672303, 500.85333787465896...","[182.0, 427.388323917137, 408.43998637602203, ...",lv,51,A4C,1FZH4296


In [23]:
file_gt_masks['file_path'][0]

'/home/ubuntu/data/01_raw/test_split50_downsampleby1000'